In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

data_list = []
missed_list1 = []
missed_list2 = []

for year in range(2020, 2024):
    for month in range(1, 13):
        month_str = str(month).zfill(2)

        for day in range(1, 32):
            day_str = str(day).zfill(2)

            url = f"https://www.tcmb.gov.tr/kurlar/{str(year)+month_str}/{day_str+month_str+str(year)}.xml"

            try:
                response = requests.get(url, timeout=10)  # Timeout süresi ekleyin
                response.raise_for_status()  # Hata durumunda istisna oluştur
            except requests.exceptions.HTTPError as errh:
                if response.status_code == 404:
                    pass
                else:
                    print(f"HTTP Hatası: {errh}")
                    print(f"Bu tarihler çekilemedi {day_str}-{month_str}-{year}")
                    missed1 = {
                        "Url": url,
                        "Reason": errh,
                        "Tarih": pd.to_datetime(day_str + month_str + str(year), format='%d%m%Y')
                    }
                    missed_list1.append(missed1)
                    time.sleep(10)  # 10 saniye bekleyin ve tekrar deneyin
                continue
            except requests.exceptions.RequestException as e:
                print(f"Hata: {e}")
                print(f"Bu tarihler çekilemedi {day_str}-{month_str}-{year}")
                missed2 = {
                    "Url": url,
                    "Reason": e,
                    "Tarih": pd.to_datetime(day_str + month_str + str(year), format='%d%m%Y')
                }
                missed_list2.append(missed2)
                time.sleep(10)  # 10 saniye bekleyin ve tekrar deneyin
                continue

            soup = BeautifulSoup(response.text, "xml")
            currencies = soup.find_all("Currency")

            for currency in currencies:
                data = {
                    "Kod": currency["Kod"],
                    "Isim": currency.find("Isim").text,
                    "ForexBuying": currency.find("ForexBuying").text,
                    "ForexSelling": currency.find("ForexSelling").text,
                    "BanknoteBuying": currency.find("BanknoteBuying").text,
                    "BanknoteSelling": currency.find("BanknoteSelling").text,
                    "CrossRateUSD": currency.find("CrossRateUSD").text if currency.find("CrossRateUSD") else None,
                    "CrossRateOther": currency.find("CrossRateOther").text if currency.find("CrossRateOther") else None,
                    "Tarih": pd.to_datetime(day_str + month_str + str(year), format='%d%m%Y')
                }
                data_list.append(data)
df = pd.DataFrame(data_list)
missedlist1 = pd.DataFrame(missed_list1)
missedlist2 = pd.DataFrame(missed_list2)
df

Hata: ('Connection aborted.', ConnectionResetError(10054, 'Varolan bir bağlantı uzaktaki bir ana bilgisayar tarafından zorla kapatıldı', None, 10054, None))
Bu tarihler çekilemedi 24-11-2021
Hata: ('Connection aborted.', ConnectionResetError(10054, 'Varolan bir bağlantı uzaktaki bir ana bilgisayar tarafından zorla kapatıldı', None, 10054, None))
Bu tarihler çekilemedi 30-01-2022
Hata: ('Connection aborted.', ConnectionResetError(10054, 'Varolan bir bağlantı uzaktaki bir ana bilgisayar tarafından zorla kapatıldı', None, 10054, None))
Bu tarihler çekilemedi 13-03-2022


,Kod,Isim,ForexBuying,ForexSelling,BanknoteBuying,BanknoteSelling,CrossRateUSD,CrossRateOther,Tarih
0,USD,ABD DOLARI,5.9478,5.9585,5.9436,5.9674,,,2020-01-02
1,AUD,AVUSTRALYA DOLARI,4.1521,4.1792,4.1330,4.2043,1.4291,,2020-01-02
2,DKK,DANİMARKA KRONU,0.89050,0.89488,0.88988,0.89693,6.6688,,2020-01-02
3,EUR,EURO,6.6630,6.6750,6.6584,6.6850,,1.1203,2020-01-02
4,GBP,İNGİLİZ STERLİNİ,7.8464,7.8873,7.8409,7.8991,,1.3215,2020-01-02
...,...,...,...,...,...,...,...,...,...
21528,QAR,KATAR RİYALİ,8.0303,8.1354,,,3.6453,,2023-12-29
21529,KRW,GÜNEY KORE WONU,0.02255,0.02285,,,1298,,2023-12-29
21530,AZN,AZERBAYCAN YENİ MANATI,17.2196,17.4449,,,1.7000,,2023-12-29
21531,AED,BİRLEŞİK ARAP EMİRLİKLERİ DİRHEMİ,7.9704,8.0747,,,3.6728,,2023-12-29


In [3]:
missedlist1

""


In [2]:
missedlist2

,Url,Reason,Tarih
0,https://www.tcmb.gov.tr/kurlar/202111/24112021...,"('Connection aborted.', ConnectionResetError(1...",2021-11-24
1,https://www.tcmb.gov.tr/kurlar/202201/30012022...,"('Connection aborted.', ConnectionResetError(1...",2022-01-30
2,https://www.tcmb.gov.tr/kurlar/202203/13032022...,"('Connection aborted.', ConnectionResetError(1...",2022-03-13


In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

day = "24"
month = "11"
year = "2021"

data_list = []


day_str = str(day).zfill(2)

url = f"https://www.tcmb.gov.tr/kurlar/{year+month}/{day_str+month+year}.xml"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "xml")
    currencies = soup.find_all("Currency")

    for currency in currencies:
        data = {
            "Kod": currency["Kod"],
            "Isim": currency.find("Isim").text,
            "ForexBuying": currency.find("ForexBuying").text,
            "ForexSelling": currency.find("ForexSelling").text,
            "BanknoteBuying": currency.find("BanknoteBuying").text,
            "BanknoteSelling": currency.find("BanknoteSelling").text,
            "CrossRateUSD": currency.find("CrossRateUSD").text if currency.find("CrossRateUSD") else None,
            "CrossRateOther": currency.find("CrossRateOther").text if currency.find("CrossRateOther") else None,
            "Tarih": pd.to_datetime(day_str + month + year, format='%d%m%Y')
        }
        data_list.append(data)
else:
    print(f"Bu tarihler çekilemedi {day_str}-{month}-{year}")

df2 = pd.DataFrame(data_list)

df2

,Kod,Isim,ForexBuying,ForexSelling,BanknoteBuying,BanknoteSelling,CrossRateUSD,CrossRateOther,Tarih
0,USD,ABD DOLARI,12.6082,12.6309,12.5994,12.6498,,,2021-11-24
1,AUD,AVUSTRALYA DOLARI,9.0774,9.1366,9.0356,9.1914,1.3857,,2021-11-24
2,DKK,DANİMARKA KRONU,1.9001,1.9094,1.8988,1.9138,6.6253,,2021-11-24
3,EUR,EURO,14.1524,14.1779,14.1425,14.1991,,1.1225,2021-11-24
4,GBP,İNGİLİZ STERLİNİ,16.8244,16.9121,16.8126,16.9374,,1.3367,2021-11-24
5,CHF,İSVİÇRE FRANGI,13.4557,13.5421,13.4355,13.5624,0.9349,,2021-11-24
6,SEK,İSVEÇ KRONU,1.3848,1.3991,1.3838,1.4024,9.0659,,2021-11-24
7,CAD,KANADA DOLARI,9.9278,9.9725,9.8910,10.0104,1.2683,,2021-11-24
8,KWD,KUVEYT DİNARI,41.3897,41.9312,40.7688,42.5602,,3.3013,2021-11-24
9,NOK,NORVEÇ KRONU,1.4112,1.4207,1.4102,1.4239,8.9125,,2021-11-24


In [9]:
df = pd.concat([df,df2])
df

,Kod,Isim,ForexBuying,ForexSelling,BanknoteBuying,BanknoteSelling,CrossRateUSD,CrossRateOther,Tarih
0,USD,ABD DOLARI,5.9478,5.9585,5.9436,5.9674,,,2020-01-02
1,AUD,AVUSTRALYA DOLARI,4.1521,4.1792,4.1330,4.2043,1.4291,,2020-01-02
2,DKK,DANİMARKA KRONU,0.89050,0.89488,0.88988,0.89693,6.6688,,2020-01-02
3,EUR,EURO,6.6630,6.6750,6.6584,6.6850,,1.1203,2020-01-02
4,GBP,İNGİLİZ STERLİNİ,7.8464,7.8873,7.8409,7.8991,,1.3215,2020-01-02
...,...,...,...,...,...,...,...,...,...
16,CNY,ÇİN YUANI,1.9627,1.9883,,,6.3880,,2021-11-24
17,PKR,PAKİSTAN RUPİSİ,0.07152,0.07246,,,175.30,,2021-11-24
18,QAR,KATAR RİYALİ,3.4432,3.4883,,,3.6412,,2021-11-24
19,KRW,GÜNEY KORE WONU,0.01056,0.01070,,,1187,,2021-11-24


In [17]:
df = df.sort_values(by="Tarih").reset_index(drop=True)

In [19]:
df.to_csv("kurlar.csv")